# Студенти основних студија у Републици Србији

У наставку ћемо истражити интересовања студената кроз отворене податке Министарства за просвету, науку и технолошки развој: http://opendata.mpn.gov.rs/index.php?ucenici_studenti=visoko. Прерађен фајл у .csv формату налази се у локалном фолдеру, а у наставку ћемо се фокусирати на: 
- интересовања студената по областима, факултетима и универзитетима, која ћемо представљати стубичастим дијаграмима
- ефикасност високог образовања - истражујући бројеве студената на завршним годинама, као и студента који дипломирају у односу на потражњу и број уписаних на високошколску институцију
- интерактивне визуелизације уз пакет plotly.express

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import cyrtranslit

Учитали смо пар нових библиотека:
- seaborn и plotly.express су као и matplotlib.pyplot библиотеке за визуализацију података, више детаља о опцијама које оне омогућавају можете видети https://seaborn.pydata.org/examples/index.html и https://plot.ly/python/plotly-express/#plotly-express
- cyrtranslit је посебно згодна библиотека када се бавите подацима који долазе из јавног сектора пошто нам она омогућава да једноставно препишемо ћирилицу у латиницу што нам значајно убрзава куцање

Крећемо као и до сада учитавањем података које ћемо у наставку анализирати:

In [ ]:
studenti = pd.read_csv('podaci/studenti_osnovnih_studija.csv')
studenti.head(2)

Видимо да нам је на располагању једна изузетно велика табела (са преко 50 колона), али и да су све колоне које садрже текстуалне податке ћириличне. Генерално то није проблем, међутим ако хоћемо да издвајамо делове табеле зависно од конкретних вредности ове колоне то ће значити да у ћелијама са кодом ове свеске морамо да се пребацујемо са ћириличног на латинично писмо што може креирати и неке грешке, па је згодно да то решимо на почетку.

Пар колона које ћемо преписати ћирилично се налазе у наставку. Библиотека *cyrtranslit* између осталог садржи и функцију *to_latin* (и њену инверзну *to_cyrillic*), која унети текст пребацује у латинично писмо. Потребно је само поред текста проследити и 'sr' којим се функцији ставља до знања да се ради о српском језику зато што функције раде и за руски и пар других језика.

In [ ]:
studenti['Univerzitet'] = [cyrtranslit.to_latin(text,'sr') for text in list(studenti['Univerzitet'])]
studenti['Naziv ustanove'] = [cyrtranslit.to_latin(text,'sr') for text in list(studenti['Naziv ustanove'])]
studenti['Naziv programa'] = [cyrtranslit.to_latin(text,'sr') for text in list(studenti['Naziv programa'])]
studenti['Polje'] = [cyrtranslit.to_latin(text,'sr') for text in list(studenti['Polje'])]
studenti['Zvanje'] = [cyrtranslit.to_latin(text,'sr') for text in list(studenti['Zvanje'])]
studenti['Nivo'] = [cyrtranslit.to_latin(text,'sr') for text in list(studenti['Nivo'])]

In [ ]:
studenti.head(2)

Колону Тип установе нисмо преименовали из следећег разлога, хајде да видимо који све типови установа постоје:

In [ ]:
studenti['Tip ustanove'].unique()

Како је у питању само једна вредност, тј сви факултети који су део овог сета података су Државни факултети, онда нам ова колона није превише корисна и можемо је избацити из посматраног сета података на следећи начин:

In [ ]:
studenti = studenti.drop(columns=['Tip ustanove'])

Преостале колоне ћемо погледати на следећи начин као и до сада:

In [ ]:
studenti.info()


Детаљнијим прегледом типова колона података, видимо да у пуно колона немамо баш све информације о бројевима студената, па ћемо пробати да групишемо и агрегирамо неке од доступних података не би ли тако стекли неку слику о високошколском образовању.

Као аргументе функције груписања, искористићемо колоне - назив установе, универзитет коме припада, ниво студија, дужину трајања програма, поље и власништво - док ћемо затим све бројчане колоне у оквиру ових група сумирати. То значи да ако неки факултет има више различитих смерова трогодишњих основних студија, нећемо бринути појединачно о студентима сваког смера, већ само о укупном броју студената по годинама:

In [ ]:
studenti_agregirano = pd.DataFrame(studenti.groupby(['Naziv ustanove','Univerzitet','Nivo','Dužina trajanja programa (u god)','Polje_skraceno','Vlasnistvo'])[['Akreditaciona kvota','Broj studenata','Broj prijavljenih kandidata','Broj svršenih studenata tokom prethodne školske godine','Studenti prva godina','Studenti druga godina','Studenti treća godina','Studenti četvrta godina','Studenti peta godina','Studenti šesta godina','Broj budžetskih studenata', 'Broj samofinansirajućih studenata',
       'Broj stranih studenata','Broj studenata na zavrsnoj godini']].sum())
studenti_agregirano.head(4)

Већ у прва 4 реда ове табеле, видимо да подаци које имамо на располагању нису потпуни, али анализираћемо оно што имамо да видимо да ли се ту крије нешто интересантно (о непотпуности, видети 4. ред по коме делује да ИМТ смер академије уметности нема ни једног студента иако их је било пријављено чак 20).

Како табела која настаје груписањем за индексе има све колоне по којима смо груписали, можемо ресетовати индекс пошто нам таква врста индексирања није неопходна тренутно:

In [ ]:
studenti_agregirano = studenti_agregirano.reset_index()
studenti_agregirano.head(2)

Овим смо велику табелу од 866 редова и 52 колоне са непотпуним уносима, свели на мало мању табелу у којој смо спојили више различитих смерова и фокусирали се само на пар најважнијих колона. Очигледно су се неки факултети понављали више пута, те задатак за знатижељне може бити бројање смерова и факултетима по универзитету (биће то мало подсећање на груписање и бројање из претходних радних свесака).

Новонасталу табелу на којој ћемо радити, можемо истражити и уз помоћ функције *describe* која нам за све бројчане колоне даје и неке додатне информације:

In [ ]:
studenti_agregirano.describe()

Из табеле коју смо добили уз помоћ груписања можете видети којих 10 факултета имају највише студената, допуните линију кода у наставку:

In [ ]:
studenti_agregirano.groupby()['Broj studenata'].sum().sort_values()[:10]

Слично, истражите којих 10 су најтраженији:

In [ ]:
#место за ваш код

Јел вас популарност неких факултета иненадила? Где се ваш факултет налази по потражњи и(ли) укупном броју студената?

За разумевање популарности факултета, није довољно да гледамо само укупан број пријављених, већ је згодно тај број ставити у контекст, зато што није исто да ли се 150 студената пријављује на факултет који прима 150 или 15 студената. Стога ћемо увести и колону која нам говори о томе колико је пријављених у односу на акредитован број места на факултету:

In [ ]:
studenti_agregirano['Potraznja'] = round(studenti_agregirano['Broj prijavljenih kandidata']/studenti_agregirano['Akreditaciona kvota'],2)

Треба имати на уму да подаци често нису потпуни, па је можда акредитациона квота негде недостајућа (0) па ће ово дељење наићи на проблем:

In [ ]:
len(studenti_agregirano[studenti_agregirano['Akreditaciona kvota']==0])

Чак 20 разматраних факултета има у пољу акредитованих студената 0! У тим случајевима, немамо адекватну процењену потражњу, погледајмо детаљније те факултете:

In [ ]:
studenti_agregirano[studenti_agregirano['Akreditaciona kvota']==0]

Испоставља се да су факултети без унете Акредитационе квоте такође и факултети који немају унете податке о пријављеним студентима, те је израчунати однос 'NaN'. Погледајмо сад који су то факултети са највећом потражњом по овом индексу:

In [ ]:
studenti_agregirano.sort_values(by='Potraznja',ascending=False)[['Naziv ustanove','Univerzitet','Potraznja']][:10]

По овом коефицијенту, борба за место на факултету је највиша на факултету драмских уметности (где се преко 8 студената бори за једно место!), мада је Математички факултет, УБ задржао своју високу позицију и када се подаци сагледају у контексту. Интересантно је што је велики број друштвених факултета који се налазе на претходној топ 10 листи, нестао а њихово место су заузели медицински факултети. Представите ове податке уз помоћ стубичастог дијаграма:

In [ ]:
#место за ваш код

Са друге стране, поред потражње, занима нас и излазност студената са факултета, то јест, колико студената заврши уписани факултет. Како немамо број студената који су уписали годину пре 3, 4 или 5 година, да адекватно проценимо који проценат студената је дипломирао, погледаћемо односе између тренутних студената 1. године и студената који су ове године завршили:

In [ ]:
studenti_agregirano['Izlaznost'] = studenti_agregirano['Studenti prva godina']/studenti_agregirano['Broj svršenih studenata tokom prethodne školske godine']

Опет морамо проверити колико факултета није унело информације о дипломираним студентима као и како су одговарајући односи израчунати у том случају (тј. који је исход недозвољеног дељења са 0).

In [ ]:
len(studenti_agregirano[studenti_agregirano['Broj svršenih studenata tokom prethodne školske godine']==0])

In [ ]:
studenti_agregirano[studenti_agregirano['Broj svršenih studenata tokom prethodne školske godine']==0]

In [ ]:
#кометар на инф

In [ ]:
studenti_agregirano[studenti_agregirano.Izlaznost!=np.inf].sort_values(by='Izlaznost',ascending=False)[['Naziv ustanove','Univerzitet','Polje_skraceno','Izlaznost']][:10]

67 пута више студената је уписало Природно-математички факултет у Новом Саду у односу на број оних који су завршили исти факултет! Погледајмо детаљније тај факултет да видимо да ли је по среди опет неки проблем непотпуне документације:

In [ ]:
studenti_agregirano[studenti_agregirano['Naziv ustanove']=='Prirodno-matematički fakultet']

Видимо да је у питању само један од нива студија који Природно математички факултет нуди и могуће је да су резултати такви зато што је у питњу нови смер, док су односи између уписаних и дипломираних студената доста бољи за остале смерове.

Како је најмање факултета који нису унели ни једног од студената, још један начин да приступимо анализи потражње и излазности са факултета је да уведемо колону процечног броја студената по години, и да њу искористимо за контекстуализовање потражње и излазности:

In [ ]:
len(studenti_agregirano[studenti_agregirano['Broj studenata']==0])

In [ ]:
studenti_agregirano['Prosecan br studenata'] = studenti_agregirano['Broj studenata']/studenti_agregirano['Dužina trajanja programa (u god)']

Друга верзија нормиране потражње и излазности:

In [ ]:
studenti_agregirano['Potraznja 2'] = studenti_agregirano['Broj prijavljenih kandidata']/studenti_agregirano['Prosecan br studenata']
studenti_agregirano['Izlaznost 2'] = studenti_agregirano['Broj svršenih studenata tokom prethodne školske godine']/studenti_agregirano['Prosecan br studenata']

Пре него што наставимо са визуелним истраживањем ових података, увешћемо још једну колону у којој ћемо пратити проценат самофинансирајућих студената.

In [ ]:
studenti_agregirano['Procenat samofinansirajucih']=np.round(100*studenti_agregirano['Broj samofinansirajućih studenata']/studenti_agregirano['Broj studenata'],2)

### Тачкасти дијаграми

Када имамо две варијабле и интересује нас да ли су оне у некој вези, једноставан начин да то тестирамо или само видимо како подаци изгледају је цртање тачкастих дијаграма. Већ кроз функцију *plot* је то могуће тиме што одаберемо да се линја не повезује, али на располагању нам се налази и функција *scatter*, која има још неке додатне опције.

(још један аргумент за цртање тачкастих дијаграма налази се овде: https://en.wikipedia.org/wiki/Anscombe%27s_quartet, а за даљу забаву препоручујем http://guessthecorrelation.com/)

Погледајмо да ли постоји веза између броја студената на факултету и броја дипломираних студената на истом факултету, погледаћемо то прво уз помоћ *plot* функције, без повезивања тачака:

In [ ]:
plt.plot(studenti_agregirano['Prosecan br studenata'],studenti_agregirano['Broj svršenih studenata tokom prethodne školske godine'],'o')
plt.show()

Исти подаци уз помоћ функције scatter изгледају:

In [ ]:
plt.scatter(studenti_agregirano['Prosecan br studenata'],studenti_agregirano['Broj svršenih studenata tokom prethodne školske godine'])
plt.show()

Функцији *scatter* није потребно наговесетити да не треба да повеже податке, а аргументом marker могуће је мењати облик тачкица. На графицима изнад, представљен је сваки од студијских програма на факултетима које анализирамо, и видимо да постоји нека веза између одабраних података. Међутим, ова веза је очекивана, на већим факултетима има више студената који би могли да заврше студије. Стога је обично битно обратити пажњу да се ове очекиване везе међу варијаблама заобиђу. На пример, овде уместо укупног броја студената који су завршили, можемо гледати колону излазност тј.однос између студената који су уписали и који су завршили факултет:

In [ ]:
plt.scatter(studenti_agregirano['Prosecan br studenata'],studenti_agregirano['Izlaznost'],s=0.1*studenti_agregirano['Broj svršenih studenata tokom prethodne školske godine'])
plt.ylim([-0.2,6])
plt.show()

Овде је већ теже рећи да постоји јасан тренд (искористили смо још једну опцију функције *scatter*, величина кружића сада одговара броју студената који су завршили факултет), мада видимо да већи факултети (десно на х оси, тј. факултети са великим бројем студената на години) имају практично константну излазност, тек сваки други уписани студент заврши. Са друге стране међу мањим факултетима имамо знатно већи распон - велики број њих има излазност већу од 1 (дакле не заврше сви уписани), али има и оних који имају излазност мању од 1, тј. више студената је у току претходне године завршило факултет него што га је уписало. То може бити последица чињенице да су неки факултети изгубили на популарности у међувремену, или да се пуно студената задржало на последњим годинама па су притиснути роковима завршили.

Следеће питање је да ли постоји веза између привлачности факултета и излазности? Да ли факултети који су најпожељнији су у исто време и изузетно ефикасни (обзиром да имају највише простора да направе селекцију и одаберу најбоље студенте, а и можда изузетна студентска жеља за тим факултетима долази помаже да се брже факултет заврши)? 
Одговор на ово питање можемо потражити укрштањем података о излазности и привлачности различитих факултета:

Слике у наставку приказују две верзије тачкастог дијаграма, једну из библиотеке *matplotlib* и библиотеке *seaborn*. У оба случаја, искористили смо опције да поред х и у координата, кружиће разликујемо и по њиховој величини која одговара просечном броју студената на факултету.

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.scatter(studenti_agregirano['Potraznja'],studenti_agregirano['Izlaznost'],s=0.2*studenti_agregirano['Prosecan br studenata'],alpha=0.5)
plt.ylim([0.1,10])
plt.xlabel('Potražnja \n Koliko studenata se takmiči za 1 mesto na fakultetu?')
plt.ylabel('Izlaznost \n Koliko puta više studenata upiše nego što završi fakultet?' )

plt.subplot(1,2,2)
sns.scatterplot(studenti_agregirano['Potraznja'],studenti_agregirano['Izlaznost'],hue=studenti_agregirano['Polje_skraceno'],s=0.2*studenti_agregirano['Prosecan br studenata'],alpha=0.5)
plt.ylim([0.1,10])
plt.legend(bbox_to_anchor=(1,1))
plt.show()

Два графика изнад приказују исте податке - свака тачкица је један факултет, њена х и у координата су Потражња и Излазност како смо их раније дефинисали, док је величина круга укупан број студената. Приказали смо исту информацију на два начина само зарад поређења тачкастих дијаграма из две различите библиотеке. Иако смо дозволили транспарентност параметром *alpha* и обојили тачкице по различитим пољима, чак и скратили распон Излазности (сетимо се, вредност ове колоне иде и до 67!) да бисмо боље сагледали различите факултете посматрање ових тачкица нам не помаже да једноставно сазнамо који су то факултети код којих је потражња велика, а ипак тек сваки нпр. сваки 5. студент успе да их заврши.

Оно што би нам значило је опција да директно очитавамо који факултети су у питању са графика. Када има овако пуно тачака, лабеловање тачака такође није опција, те је спас у интерактивним графицима у којима ћемо преласком мишем преко тачке од интереса долазити до додатних информација.

In [ ]:
fig = px.scatter(studenti_agregirano.dropna(), x="Potraznja", y="Izlaznost", color="Polje_skraceno",
                 size='Broj studenata', 
                 hover_name="Naziv ustanove"               
                 , range_x=[-0.5,5]
                 , range_y=[0.1,10]
                )
fig.show()

Поновили смо све остале параметре графика као у статичној верзији да бисмо што лакше уочили промене у новом окружењу. Сада преласком преко било ког кужића сазнајемо о ком факултету се ради, колико студената укупно има, која је потражња, а која излатзност.
Интересантно је да је најтраженији факултет по броју студената који аплицирају на једно место - Факултет драмских уметности (доле десно), такође и факултет на коме је излазност доста блиска јединици, што је доста похвално, али на жалост није тренд. Већ други по потражњи - Математички факултет - одликује се чињеницом да тек сваки 5 студент заврши! Можемо додатно сузити опсеге х и у оса, а можемо и обојити тачкице по универзитетима уместо по пољима да видимо да ли нам је тако ситуација јаснија.

In [ ]:
fig = px.scatter(studenti_agregirano, x="Potraznja", y="Izlaznost", color="Univerzitet",
                 size='Broj studenata', 
                 hover_name="Naziv ustanove"
                 , range_x=[-0.5,4]
                 , range_y=[0.3,8]
                )
fig.show()

Интересантно је да постоје факултети код којих је потражња мала, не долази до испуњавања акредитованих места, што би значило да су још већи ресурси на располагању студентима који се на њих уписују (више времена професора, бољи однос између броја студената и наставника и сл.), али се на њима и даље дешава да много мање студената завршава него што уписује (нпр факултети у левом горњем углу). Мала потражња за оваквим факултетима је можда и последица информација које се шире међу студентима о тежини ових факултета.

Наредно питање које се поставља тиче се опет факултета са малом излазношћу, шта се дешава са студентима који не дипломирају? Да ли ти студенти одустају од студија на вишим годинама, па је број студената све мањи и мањи до дипломирања или је малом броју диплома узрок то што се студенти "заглаве" на некој од година студирања. За ову информацију можемо погледати однос између броја студената на завршној и првој години студија.

In [ ]:
studenti_agregirano['Zagusenje'] = studenti_agregirano['Broj studenata na zavrsnoj godini']/studenti_agregirano['Studenti prva godina']

In [ ]:
fig = px.scatter(studenti_agregirano, x="Zagusenje", y="Izlaznost", color="Univerzitet",
                 size='Broj studenata', 
                 hover_name="Naziv ustanove"
                 , range_x=[-0.5,4.5]
                 , range_y=[0.3,8]
                )
fig.show()

Интересантно је да нису факултети са најлошијом стопом излазности они на којима је загушење студената највише. Код њих је број студената на завршној години студија мањи него на првој, што говори о томе да се студенти вероватно самоелиминишу некада раније. Са друге стране, пуно је факултета на којима тек сваки други или трећи студент прве године дипломира, али их је на завршној години и до 2,3, или 4 пута више него на првој години. Овде се поставља и питање капацитета факултета да се избори са толиким бројем студената. На следећем графику можемо проверити да ли су факултети са 3-4 пута више студената на завршној години уписали мање студената него што је акредитовано, те можда имају капацитета да се изборе, или су напротив, на завршној години студија дошли у ситуацију да имају многоструко више студената него што могу да поднесу.

In [ ]:
studenti_agregirano['Popunjenost mesta']=studenti_agregirano['Studenti prva godina']/studenti_agregirano['Akreditaciona kvota']

In [ ]:
fig = px.scatter(studenti_agregirano, x="Popunjenost mesta", y="Zagusenje", color="Univerzitet",
                 size='Broj studenata', 
                 hover_name="Naziv ustanove"
                 , range_x=[-0.5,4.5]
                 , range_y=[0.3,8]
                )
fig.show()

На овом графику је интересантно истражити све тачкице код којих је попуњеност места у близини јединице (значи да су они своју квоту испунили или надмашили!), а код којих је загушење веће од 1. Један такав пример је Економски факултет (у близини координата 1.5,2.5), на који се упише више од акредитоване квоте, али и поред тога на завршној години студија има и преко два пута више студената него што их је било на првој години. 

In [ ]:
studenti_agregirano['Premasenje kvote']=studenti_agregirano['Broj studenata na zavrsnoj godini']/studenti_agregirano['Akreditaciona kvota']

In [ ]:
fig = px.scatter(studenti_agregirano, x="Premasenje kvote", y="Izlaznost", color="Univerzitet",
                 size='Broj studenata', 
                 hover_name="Naziv ustanove"
                 , range_x=[-0.5,4.5]
                 , range_y=[0.3,5]
                )
fig.show()

Изгледа да се за значајан део факултета лоша излазност (тј. велики однос између броја уписаних и дипломираних приказан на у оси) може објаснити преоптерећеношћу факултета, односно премашењем акредитационе квоте (приказано на х оси као однос броја студената на завршној години и броја акредитованих места).

Издвојте факултете за којима влада велика потражња, али је излазност лоша:

**Задатак**

Да ли факултети за којима влада велика потражња уписују више студената па тиме удео самофинансирајућих студената расте? Да ли је удео самофинансирајућих студената у вези са већим загушењем на факултету (па студенти нужно прелазе на самофинансирање) или факултети са више самофинансирајућих студената имају бољу излазност?
Потражите одговоре на ова питања одговарајућим тачкастим дијаграмом, фокусирајте се само на факултете код којих је власништво државно.
Нацртајте следеће дијаграме:
- стубичасти дијаграм за 10 факултета са највећим односом између самофинансирајућих и буџетских студената
- пар различитих тачкастих дијаграма на којима ће једна оса бити новоуведена колона (однос самофиснансирајућих и буџетских студената), а за другу осу ћете користи неке од претходно израчунатих колона попут потражње за факултетом, пролазности и загушења